# 🤖 Assistente de Voz Multi-Idiomas Com GEMINI

---

In [1]:
# Instalação das dependências
!pip install -q -U google-generativeai gTTS

In [2]:
# Bibliotecas
import google.generativeai as genai
from IPython.display import Audio, display, Javascript
from google.colab import output, userdata
from base64 import b64decode
from gtts import gTTS
import time
import os

/usr/local/lib/python3.12/dist-packages/google/colab/_import_hooks/_hook_injector.py:55: FutureWarning: 

All support for the `google.generativeai` package has ended. It will no longer be receiving 
updates or bug fixes. Please switch to the `google.genai` package as soon as possible.
See README for more details:

https://github.com/google-gemini/deprecated-generative-ai-python/blob/main/README.md

  loader.exec_module(module)


## ⚙ Configuração do Modelo Gemini

In [3]:
try:
    # Busca a chave salva nos "Secrets" do Colab
    GOOGLE_API_KEY = userdata.get('GEMINI_API_KEY')
    genai.configure(api_key=GOOGLE_API_KEY)
    # Usamos o 1.5-flash-latest por ser o mais estável para áudio
    model = genai.GenerativeModel('gemini-2.5-flash')
    print("✅ Sistema pronto e conectado ao Gemini.")
except Exception as e:
    print(f"❌ Erro de configuração: {e}")

✅ Sistema pronto e conectado ao Gemini.


## 🧹 Limpa Arquivos Existentes de Áudio

In [4]:
# Função para limpar arquivos existentes
def clean_files(file_list):
    for file in file_list:
        if os.path.exists(file):
            os.remove(file)
            print(f"🗑️ Arquivo antigo removido: {file}")

## 🎤 Gravação de Áudio com Python e JavaScript

In [5]:
# Lógica de gravação utilizando JavaScript
RECORD_JS = """
const sleep = time => new Promise(resolve => setTimeout(resolve, time));
const b2text = blob => new Promise(resolve => {
  const reader = new FileReader();
  reader.onloadend = e => resolve(e.srcElement.result);
  reader.readAsDataURL(blob);
});
var record = time => new Promise(async resolve => {
  const stream = await navigator.mediaDevices.getUserMedia({ audio: true });
  const recorder = new MediaRecorder(stream);
  const chunks = [];
  recorder.ondataavailable = e => chunks.push(e.data);
  recorder.start();
  await sleep(time);
  recorder.onstop = async () => {
    const blob = new Blob(chunks);
    const text = await b2text(blob);
    resolve(text);
  };
  recorder.stop();
});
"""

## 🧠 Reconhecimento de Fala e Integração com Gemini

In [6]:
# Função para iniciar gravação e conectar ao Gemini
def start_assistant(sec=5):
    USER_AUDIO = 'user_input.wav'
    BOT_AUDIO = 'bot_response.mp3'

    clean_files([USER_AUDIO, BOT_AUDIO])

    try:
        # Gravação
        display(Javascript(RECORD_JS))
        print(f"🎤 Gravando ({sec}s)... Fale agora!")
        js_result = output.eval_js(f'record({sec * 1000})')
        audio_data = b64decode(js_result.split(',')[1])

        with open(USER_AUDIO, 'wb') as f:
            f.write(audio_data)

        # Processamento Gemini
        print("📤 Enviando áudio...")
        uploaded_file = genai.upload_file(path=USER_AUDIO)

        while uploaded_file.state.name == "PROCESSING":
            time.sleep(1)
            uploaded_file = genai.get_file(uploaded_file.name)

        print("🧠 Analisando sua pergunta...")

        # --- NOVO: Pedindo a transcrição e a resposta juntas ---
        prompt = (
            "Primeiro, transcreva exatamente o que foi dito no áudio. "
            "Depois, forneça uma resposta amigável em português. "
            "Formate assim:\n"
            "VOCÊ DISSE: [transcrição]\n"
            "GEMINI: [resposta]"
        )

        response = model.generate_content([prompt, uploaded_file])

        # Exibe o texto completo (Pergunta + Resposta)
        print("-" * 30)
        print(response.text)
        print("-" * 30)

        # Para converter em voz apenas a parte da resposta do Gemini
        # Vamos tentar separar o texto para o áudio não ler "Você disse..."
        texto_para_voz = response.text.split("GEMINI:")[-1].strip()

        # Voz de saída
        tts = gTTS(text=texto_para_voz, lang='pt')
        tts.save(BOT_AUDIO)
        display(Audio(BOT_AUDIO, autoplay=True))

    except Exception as e:
        print(f"❌ Erro: {e}")

#  --- EXECUTAR ---
start_assistant(sec=6)

🗑️ Arquivo antigo removido: user_input.wav
🗑️ Arquivo antigo removido: bot_response.mp3


<IPython.core.display.Javascript object>

🎤 Gravando (6s)... Fale agora!
📤 Enviando áudio...
🧠 Analisando sua pergunta...
------------------------------
VOCÊ DISSE: O que é e para que serve o Gemini?
GEMINI: Olá! Eu sou o Gemini, um modelo de linguagem grande treinado pelo Google. Fui projetado para te ajudar com uma variedade de tarefas, como responder às suas perguntas, gerar textos criativos, traduzir idiomas, resumir informações, escrever códigos e muito mais! Basicamente, estou aqui para processar informações e interagir com você de forma útil e informativa. No que mais posso ajudar hoje?
------------------------------
